<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Indoor Sensors</b></header>

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>Introduction</b></p>


<p style = 'font-size:16px;font-family:Arial'>People often think of GeoSpatial as related to latitude, longitude and occasionally altitude, but it can also be applied to locations within a space such as a building. When you apply a time dimension to location,  you get “4D Analytics”.  In this demo, we will be using data from 54 building sensor locations at the Intel Berkeley Research lab. They produce an average of 68,000 readings per day over the month used in this demo. We will not only analyze the temperature, humidity, and light but also the connectivity of the sensors to each other and voltage readings.</p> 

<p style = 'font-size:16px;font-family:Arial'>The complexity of the analysis is that the sensors produce information approximately every 31 seconds, but not all sensors report at exactly the same time and some sensors will have occasional outages during the day due to communications or voltage issues.  This type of irregular data presence happens in multiple industries. Examples: a retail POS system may be late in communicating due to store system issues or warehouse inventories may be taken at irregular intervals. For this complexity, we will use Vantage Time Series Analytics capabilities.</p>

<p style = 'font-size:16px;font-family:Arial'>This sensor data is an example of high volume, IoT data that is best left on low cost cloud storage.  When running this demo in the ClearScape Analytics Experience, the Vantage Database running on Google Cloud is accessing IoT data that is stored on AWS S3.</p>


<p style = 'font-size:16px;font-family:Arial'>We will connect to Vantage and explore a sensor dataset provided by the <b>Intel Berkeley Research Lab</b>.</p>


<b style = 'font-size:20px;font-family:Arial;color:#E37C4D'>1. Connect to Vantage.</b>
<p style = 'font-size:16px;font-family:Arial'>We start by importing the required libraries and connecting to the database.</p>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import getpass
import networkx as nx


from teradataml import *
import warnings
warnings.filterwarnings('ignore')

<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, then use down arrow to go to next cell. Begin running steps with Shift + Enter keys.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=Sensor_Data_Analytics_PY_SQL.ipynb;' UPDATE FOR SESSION; ''')

<hr>
<b style = 'font-size:20px;font-family:Arial;color:#E37C4D'>2. Getting Data for This Demo </b>

<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage.  You have the option of either running the demo using foreign tables to access the data without using any storage on your environment or downloading the data to local storage which may yield somewhat faster execution, but there could be considerations of available storage.  There are two statements in the following cell, and one is commented out.  You may switch which mode you choose by changing the comment string. </p>


In [ ]:
%run -i ../run_procedure.py "call get_data('DEMO_IndoorSensor_cloud');"
 # Takes about 20 seconds
# %run -i ../run_procedure.py "call get_data('DEMO_IndoorSensor_local');" 
# Takes about 40 seconds

<p style = 'font-size:16px;font-family:Arial'>Next is an optional step – if you want to see status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>3. Check sensor locations</b>
<p style = 'font-size:16px;font-family:Arial'>We have the lab sensor locations already loaded into Vantage, so load them into a teradataml DataFrame:

In [ ]:
mote_x_y = DataFrame(in_schema('DEMO_IndoorSensor', 'sensor_locations'))

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>4. Plot Sensor Locations in 2D Floor Plan</b></p>

<p style = 'font-size:16px;font-family:Arial'>Let's make sure our data matches the Intel provided map of the sensor locations as below:</p>

<p style = 'font-size:16px;font-family:Arial'><b> Sensor Location Map</b></p>

<img src="images/intel_lab_1.png">

<p style = 'font-size:16px;font-family:Arial'>Note default plot origin(0,0) is bottom left, while in the Intel lab diagram, origin is top right. Therefore we need to flip the origin when plotting.</p>

In [ ]:
qry = '''create multiset table sensor_new_locations as (select id, 41-x x, 32-y y from "DEMO_IndoorSensor"."sensor_locations") with data;'''


try:
    execute_sql(qry)
except:
    db_drop_table('sensor_new_locations')
    execute_sql(qry)

df=DataFrame.from_query('SELECT Top 5 * FROM sensor_new_locations;')
df.sort(['x','y'], ascending= True)


In [ ]:
mote_x_y = DataFrame('sensor_new_locations')

In [ ]:
mote_x_y_pandas = mote_x_y.to_pandas().reset_index()   # Get a pandas DataFrame for plotting

fig, ax = plt.subplots(figsize=(16, 9))

for x, y, z in zip(mote_x_y_pandas.x, mote_x_y_pandas.y, mote_x_y_pandas.id):
    ax.annotate(z, (x, y))

ax.scatter(mote_x_y_pandas.x, mote_x_y_pandas.y)
 
plt.show()

<p style = 'font-size:16px;font-family:Arial'>Looks good! Now let's explore the connectivity between the sensors...</p>

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>5. Check Sensor Communications</b></p>

<p style = 'font-size:16px;font-family:Arial'>We have the connectivity between the sensors also loaded into Vantage:

In [ ]:
connectivity = DataFrame(in_schema('DEMO_IndoorSensor', 'connectivity'))

In [ ]:
connectivity_pandas = connectivity.to_pandas().reset_index()
connectivity_pandas.sort_values(by='reachprob', ascending=False)

<p style = 'font-size:16px;font-family:Arial'>Clean up and remove some of the null data</p>

In [ ]:
connectivity_clean = connectivity_pandas.query('sendid != 0')
len(connectivity_clean.index)

In [ ]:
connectivity_clean = connectivity_clean.query('receiveid != 0')
len(connectivity_clean.index)

In [ ]:
connectivity_clean = connectivity_clean.query('reachprob != 0')
len(connectivity_clean.index)

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>6. Visualize Sensor Communications</b></p>
<p style = 'font-size:16px;font-family:Arial'>Build connectivity network:</p>

In [ ]:
DG = nx.DiGraph()
DG.add_weighted_edges_from(zip(connectivity_clean.sendid, connectivity_clean.receiveid, connectivity_clean.reachprob))

In [ ]:
# Define node positions data structure (dict) for plotting, nodes positions shown as sensor locations.
pos = mote_x_y_pandas.set_index('id').T.to_dict('list')

# Preview of node_positions with a bit of hack (there is no head/slice method for dictionaries).
dict(list(pos.items())[0:5])

<p style = 'font-size:16px;font-family:Arial'><b>Use reachprob as weight, which is the probability of sensorA's msg sent to sensorB.</b></p>

<ol style = 'font-size:16px;font-family:Arial'> 
    <li>reachprob>0.5 shown in red</li>
    <li>0.5>=reachprob>0.1 shown in blue</li>
    <li>0.1>=reachprob shown in green</li> 
</ol>
<p style = 'font-size:16px;font-family:Arial'><b>Node size reflects the sensor's capability of sending/receiving messages from other sensors. Bigger node size implies better communication.</b></p>

<p style = 'font-size:16px;font-family:Arial'>Below code creates a graph where Nodes are Sensors and edges are assigned according to the strength of the signal(reachprob).</p>

In [ ]:
plt.figure(figsize=(16, 9))

elarge = [(u, v) for (u, v, d) in DG.edges(data=True) if d['weight'] > 0.5]
esmall1 = [(u, v) for (u, v, d) in DG.edges(data=True) if ((d['weight'] > 0.1) and (d['weight'] <= 0.5))]
esmall2 = [(u, v) for (u, v, d) in DG.edges(data=True) if d['weight'] <= 0.1]

# nodes
d = dict(DG.degree(weight='weight'))
nx.draw_networkx_nodes(DG, pos, nodelist=d.keys(), node_size=[ v*100 for v in d.values()], 
                       node_color='white', edgecolors='black')

# edges
nx.draw_networkx_edges(DG, pos, edgelist=elarge, arrowstyle="->", arrowsize=1,
                       width=0.5, edge_color='r')
nx.draw_networkx_edges(DG, pos, edgelist=esmall1, arrowstyle="->", arrowsize=1,
                       width=0.2, alpha=0.5, edge_color='b', style='dashed')
nx.draw_networkx_edges(DG, pos, edgelist=esmall2, arrowstyle="->", arrowsize=1,
                       width=0.05, alpha=0.5, edge_color='g', style='dashed')

# labels
nx.draw_networkx_labels(DG, pos, font_size=15, font_family='sans-serif')

plt.axis('off')
plt.show()

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>7. Get Sensor Readings Data from the Object Store</b></p>

<p style = 'font-size:16px;font-family:Arial'>Sensors and other IOT devices generate <b>a lot</b> of data. In our case the sensor data is located in an object store because of the cost/performance tradeoff and resilience it provides.</p>

<p style = 'font-size:16px;font-family:Arial'> We will do the following steps to get data from Object Store</p>
<li style = 'font-size:16px;font-family:Arial'>Define a schema providing the column names and the delimiter.</li>
<li style = 'font-size:16px;font-family:Arial'>Create a foreign table using the schema.</li>
<li style = 'font-size:16px;font-family:Arial'>Define a view over the foreign table  which defines the data types and formats of the columns in the CSV data.</li> 
<li style = 'font-size:16px;font-family:Arial'>Define a view that joins the view above to the sensor locations to eliminate data that is not in the lab being analyzed.</li> 



<p style = 'font-size:14px;font-family:Arial'><b>Create the Foreign Table</b></p>

<p style = 'font-size:16px;font-family:Arial'>Now create a Foreign Table to allow easy access to the raw sensor data:</p>

In [ ]:
query='''CREATE MULTISET FOREIGN TABLE sensors_table ,FALLBACK ,
     MAP = TD_MAP1
     (
      Location VARCHAR(2048) CHARACTER SET UNICODE CASESPECIFIC,
      sensdate DATE FORMAT 'YYYY-MM-DD',
      senstime TIME(6),
      epoch BIGINT,
      moteid INTEGER,
      temperature FLOAT,
      humidity FLOAT,
      light FLOAT,
      voltage FLOAT,
      sensdatetime TIMESTAMP(6))
USING
(
      location  ('/s3/s3.amazonaws.com/clearscape-analytics-datasets/sensor_readings/')
      MANIFEST  ('FALSE')
      PATHPATTERN  ('$var1/$var2')
      STOREDAS  ('PARQUET')
)
;'''

try:
    execute_sql(query)
except:
    db_drop_table('sensors_table')
    execute_sql(query)                  

<p style = 'font-size:16px;font-family:Arial'>Here is what the raw data looks like - note each row is coming back into a single 'Payload' column, we will address this in the next step...</p>

In [ ]:
sensors_table = DataFrame('sensors_table');

<p style = 'font-size:16px;font-family:Arial'>Take a sample of a few rows (10):</p>

In [ ]:
sensors_table.sample(10)

<p style = 'font-size:14px;font-family:Arial'><b>Create a View for Easy Access</b></p>

<p style = 'font-size:16px;font-family:Arial'>Create a view on top of the raw data that puts structure and proper datatypes on the sensor data:</p>

In [ ]:
execute_sql('''REPLACE VIEW sensor_readings
  AS 
    (SELECT
      CAST("sensdate" AS DATE ) sensdate,
      CAST("senstime" AS TIME(6)) senstime,
      CAST(epoch AS BIGINT) epoch,
      CAST(moteid AS INTEGER) moteid,
      CAST(temperature AS FLOAT) ( FORMAT '-ZZZ9.99') temperature,
      CAST(humidity AS FLOAT) ( FORMAT '-ZZZ9.99') humidity,
      CAST(light AS FLOAT) ( FORMAT '-ZZZ9.99') light,
      CAST(voltage AS FLOAT) ( FORMAT '-ZZZ9.99') voltage,
      cast(cast("sensdate" as varchar(10)) || ' ' || cast("senstime" as varchar(15)) AS TIMESTAMP(6)) sensdatetime
  FROM sensors_table);''')

<p style = 'font-size:16px;font-family:Arial'>Now take a look at the formatted data:</p>

In [ ]:
sensor_readings = DataFrame('sensor_readings');

In [ ]:
sensor_readings.sample(10)

<p style = 'font-size:14px;font-family:Arial'><b>Final View - Clean Up Data and Join</b></p>

<p style = 'font-size:16px;font-family:Arial'>Clean up the data a little - limit the sensorid (moteid) to 54 because there are supposed to only be 54 sensors.</p>

<p style = 'font-size:16px;font-family:Arial'>We also want to join with our local data table that gives us the location of each sensor:</p>

In [ ]:
execute_sql('''REPLACE VIEW sensor_clean AS ( 
    SELECT * FROM sensor_readings sr 
    LEFT JOIN DEMO_IndoorSensor.sensor_locations sl
    ON sr.moteid = sl.id
    WHERE sr.moteid <= 54
);''')

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>8. Time Series Analysis</b></p>

<p style = 'font-size:16px;font-family:Arial'>Our dataset is from a wireless sensor network composed of 54 sensors monitoring temperature, humidity, lighting conditions of the surrounding environment as well as voltage of each sensor. Each sensor monitors and submits a package containing the above information once every 31 s. Often times we will want to look at data in different time frames, this may be far more granular than we need for instance.</p>

<p style = 'font-size:16px;font-family:Arial'>This is where the powerful Time Series functionality of Vantage comes in. Using the GROUP BY TIME we can easily group this into 1hr increments... But this could be by minute, day, whatever time interval you choose!</p>

In [ ]:
qry = '''
SELECT 
$TD_TIMECODE_RANGE AS T_RANGE 
, $TD_GROUP_BY_TIME AS T_GROUP 
, moteid
, AVG(temperature) AS avg_temperature 
, AVG(humidity) AS avg_humidity
, AVG(light) AS avg_light
, AVG(voltage) AS avg_voltage
, AVG(x) AS x
, AVG(y) AS y
FROM 
sensor_clean
GROUP BY TIME (HOURS(1) AND moteid)                                          
USING TIMECODE(sensdatetime)
ORDER BY 1, 3;
'''
data1hr = pd.read_sql(qry, eng)

<p style = 'font-size:16px;font-family:Arial'>Vantage has generated the RANGE and GROUP columns for us to identify the timeslot the data is for. You can iterate and change the way the time series sensor data is aggregated. Try changing HOURS to DAYS and see the results. </p>

<p style = 'font-size:16px;font-family:Arial'>Now lets group the data again by Sensor id to see a summary of all 54 sensors:</p>

In [ ]:
data1hr_grp = data1hr.groupby('moteid')

In [ ]:
# review first group
data1hr_grp.first()

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>9. Visualise the Hourly Sensor Readings</b></p>

<p style = 'font-size:16px;font-family:Arial'>Now that we have used Vantage to bring in sensor data from an object store, clean it up, join it with additional data and change the time interval to hourly - lets take a look at it!</p>

<ol style = 'font-size:16px;font-family:Arial'>
    <li>Temperature is in degrees Celsius.</li>
    <li>Humidity is temperature corrected relative humidity, ranging from 0-100%.</li>
    <li>Light is in Lux (a value of 1 Lux corresponds to moonlight, 400 Lux to a bright office, and 100,000 Lux to full sunlight.)</li>
    <li>Voltage is expressed in volts.</li>

</ol>

In [ ]:
# sens_type can be avg_temperature, avg_humidity, avg_light,  or avg_voltage.
sens_type = "avg_temperature"

In [ ]:
# Note that not all 54sensors have readings in every time range, 
# therefor need to get the list of keys from the group.
fig, ax = plt.subplots(figsize=(16, 10))

plt.ylabel(sens_type)

for i in data1hr_grp.groups.keys():
    plt.plot(data1hr_grp.get_group(i).T_GROUP, 
             data1hr_grp.get_group(i)[sens_type])
    
plt.xticks(rotation='vertical')
#plt.legend(list(data1hr_grp.groups.keys()))
plt.show;

<p style = 'font-size:16px;font-family:Arial'><b>Try changing the sensor reading from temperature to another sensor (humidity, light, voltage) and see the results. You can also change the time frame aggregation from an hour to something different and see the changes.</b></p>

<p style = 'font-size:16px;font-family:Arial'>As in the above plotted graph, average temperature is plotted, we can plot average humidity, average light or average voltage. In the cell where we are assigning sens_type</p>

<p style = 'font-size:16px;font-family:Arial'># sens_type can be avg_temperature, avg_humidity, avg_light,  or avg_voltage.
    
sens_type = "avg_temperature"
</p>
    
<p style = 'font-size:16px;font-family:Arial'>Can be changed to
    
sens_type = "avg_humidity"</p>
    
<p style = 'font-size:16px;font-family:Arial'> This will help plotting the above graph using average humidity, average light or average voltage.</p>  

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>10. Cleanup </b></p>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>Cleanup work tables to prevent errors next time.</p>

In [ ]:
tables = ['sensor_new_locations', 'sensors_table']

# Loop through the list of tables and execute the drop table command for each table
for table in tables:
    try:
        db_drop_table(table_name = table)
    except:
        pass

In [ ]:
execute_sql('DROP view sensor_readings;')

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_IndoorSensor');" 
#Takes 5 seconds

In [ ]:
remove_context()

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>11. Dataset</b></p>


<p style = 'font-size:16px;font-family:Arial;'>The <b>sensor_readings</b> dataset contains 2.3 Million records - the output of 54 sensors between February 28th and April 5th, 2004:</p>


- `date`: date of the sensor reading
- `time`: time of the sensor reading
- `epoch`: store identifier where the order was taken
- `moteid`: unique sensor identifier
- `temperature`: temperature
- `humidity`: humidity
- `light`: light
- `voltage`: light

    
<p style = 'font-size:16px;font-family:Arial'>
<b>sensor_locations</b> contains xy coordinates of sensors in meters relative to the upper right corner of the lab

- `id`: sensor identifier
- `x`: X coordinate (M)
- `y`: Y coordinate (M)
</p>
    
<p style = 'font-size:16px;font-family:Arial'>
<b>connectivity</b> the sensors are wirelessly interconnected and due to locations / environment the connections vary in strength 

- `sendid`: sensor id of sender
- `receiveid`: sensor id of receiver
- `reachprob`: probability of a message from a sender successfully reaching a receiver
</p>

<p style = 'font-size:16px;font-family:Arial'>This data, the Intel Berkeley Research Lab data set, was collected through the hard work of: Peter Bodik, Wei Hong, Carlos Guestrin, Sam Madden, Mark Paskin, and Romain Thibaux. Mark aggregated the raw connectivity information over time and generated the beautiful network layout diagram. Sam and Wei wrote TinyDB and other software for logging data. Intel Berkeley provided hardware. The TinyOS team, in particular Joe Polastre and Rob Szewczyk, provided the software infrastructure and hardware designs that made this deployment possible.

http://db.csail.mit.edu/labdata/labdata.html
    </p>

<footer style="padding:10px;background:#f9f9f9;border-bottom:3px solid #394851">Copyright © Teradata Corporation - 2023. All Rights Reserved.</footer>